<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/upscale_pk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.onnx
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet

# --- レイヤー定義（Flatten → FC → Reshape） ---
class IdentityFC(nn.Module):
    def __init__(self, shape, random_init=False):
        super().__init__()
        self.shape = shape  # (C, H, W)
        flat_dim = shape[0] * shape[1] * shape[2]
        self.fc = nn.Linear(flat_dim, flat_dim, bias=False)

        if random_init:
            self.init_random()
        else:
            self.init_identity(flat_dim)

    def init_identity(self, dim):
        with torch.no_grad():
            weight = torch.zeros((dim, dim))
            for i in range(dim):
                weight[i, i] = 1.0
            self.fc.weight.copy_(weight)

    def init_random(self):
        nn.init.kaiming_normal_(self.fc.weight, a=0.01)

    def forward(self, x):
        b, c, h, w = x.shape
        x_flat = x.view(b, -1)
        x_fc = self.fc(x_flat)
        return x_fc.view(b, c, h, w)

# --- Real-ESRGANモデルにFCレイヤーを挿入 ---
def insert_fc_into_pretrained(model: nn.Module, random_init=False, verbose=True):
    dummy = torch.randn(1, 3, 1200, 1600)
    with torch.no_grad():
        x = model.conv_first(dummy)
        min_area = x.shape[2] * x.shape[3]
        min_idx = -1
        feature_maps = []

        for i, layer in enumerate(model.body):
            x = layer(x)
            area = x.shape[2] * x.shape[3]
            feature_maps.append(x)
            if area < min_area:
                min_area = area
                min_idx = i

    if verbose:
        print(f"🔍 最小特徴マップ位置: model.body[{min_idx}]、サイズ: {feature_maps[min_idx].shape}")

    before = list(model.body.children())[:min_idx + 1]
    after = list(model.body.children())[min_idx + 1:]

    fc_layer = IdentityFC(shape=feature_maps[min_idx].shape[1:], random_init=random_init)
    model.body = nn.Sequential(*before, fc_layer, *after)
    return model

# --- ONNXファイルに変換 ---
def export_to_onnx(model, input_tensor, onnx_path="exported_model.onnx"):
    model.eval()
    torch.onnx.export(model, input_tensor, onnx_path,
                      input_names=['input'], output_names=['output'],
                      dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}},
                      opset_version=11)
    print(f"✅ ONNXとして保存されました: {onnx_path}")

# --- モデルを学習（random_init=True の場合） ---
def train_model(model, target_model, epochs=1, lr=1e-4):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    dummy_input = torch.randn(1, 3, 1200, 1600)

    with torch.no_grad():
        target_output = target_model(dummy_input)

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = loss_fn(output, target_output)
        loss.backward()
        optimizer.step()
        print(f"🧪 Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

# --- GUIで結果を比較（OpenCV） ---
def visualize_output(output1, output2):
    def tensor_to_cv(img):
        img = img.squeeze().permute(1, 2, 0).clamp(0, 1).cpu().numpy()
        img = (img * 255).astype(np.uint8)
        return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    img1 = tensor_to_cv(output1)
    img2 = tensor_to_cv(output2)
    combined = np.hstack((img1, img2))
    cv2.imshow('左: Pretrained, 右: FC付き', combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# --- メイン関数 ---
def main():
    # モデル読み込み
    model_path = 'pretrained/RealESRGAN_x1_fixed_1600x1200.pth'
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                    num_block=23, num_grow_ch=32, scale=1)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()

    # FC付きモデル作成（ランダム or アイデンティティ初期化）
    use_random_init = True  # ← Trueの場合、訓練も行われる
    model_fc = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                       num_block=23, num_grow_ch=32, scale=1)
    model_fc.load_state_dict(torch.load(model_path), strict=True)
    model_fc = insert_fc_into_pretrained(model_fc, random_init=use_random_init, verbose=True)

    # 入力データ生成
    input_img = torch.randn(1, 3, 1200, 1600)

    # 学習（必要な場合のみ）
    if use_random_init:
        print("⚙️ FCレイヤーのランダム初期化に対して微調整を行います...")
        train_model(model_fc, model, epochs=3)

    # 出力計算
    with torch.no_grad():
        out1 = model(input_img)
        out2 = model_fc(input_img)
        is_same = torch.allclose(out1, out2, atol=1e-6)
        print(f"✅ 出力一致: {is_same}")

    # .pthファイルとして保存
    torch.save(model_fc.state_dict(), "modified_model_fc.pth")
    print("✅ FC付きモデルを保存しました: modified_model_fc.pth")

    # ONNX形式として保存
    export_to_onnx(model_fc, input_img, onnx_path="modified_model_fc.onnx")

    # GUIで出力画像を比較
    visualize_output(out1, out2)

if __name__ == "__main__":
    main()

In [ ]:
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from insert_fc import insert_fc_into_pretrained

def test_model_with_fc():
    # 学習済みモデルの読み込み
    model_path = 'pretrained/RealESRGAN_x1_fixed_1600x1200.pth'
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                    num_block=23, num_grow_ch=32, scale=1)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()

    # モデルを複製して、FCレイヤーを挿入
    model_fc = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                       num_block=23, num_grow_ch=32, scale=1)
    model_fc.load_state_dict(torch.load(model_path), strict=True)
    model_fc = insert_fc_into_pretrained(model_fc, random_init=False, verbose=True)
    model_fc.eval()

    # テスト用画像（ランダム）
    input_img = torch.randn(1, 3, 1200, 1600)

    # 出力を比較
    with torch.no_grad():
        out1 = model(input_img)
        out2 = model_fc(input_img)
        is_same = torch.allclose(out1, out2, atol=1e-6)

    print(f"✅ 出力が一致するか: {is_same}")

if __name__ == "__main__":
    test_model_with_fc()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from collections import OrderedDict
import copy
import os
from PIL import Image
import torchvision.transforms as transforms
from pathlib import Path

class IdentityFullyConnected(nn.Module):
    """
    空間情報を維持しながらidentity mappingを実行するFC layer
    """
    def __init__(self, num_channels, height, width, init_type='identity'):
        super(IdentityFullyConnected, self).__init__()
        self.num_channels = num_channels
        self.height = height
        self.width = width
        self.total_size = num_channels * height * width

        # Fully connected layer
        self.fc = nn.Linear(self.total_size, self.total_size, bias=False)

        # 重み初期化
        self.init_weights(init_type)

    def init_weights(self, init_type='identity'):
        """
        重み初期化関数
        Args:
            init_type: 'identity' または 'random'
        """
        with torch.no_grad():
            if init_type == 'identity':
                # Identity matrix初期化（対角線1、それ以外0）
                self.fc.weight.data = torch.eye(self.total_size)
            elif init_type == 'random':
                # Xavier uniform初期化
                nn.init.xavier_uniform_(self.fc.weight)
            else:
                raise ValueError("init_type must be 'identity' or 'random'")

    def reinit_weights(self, init_type='identity'):
        """重み再初期化関数"""
        self.init_weights(init_type)

    def forward(self, x):
        batch_size = x.size(0)

        # (B, C, H, W) -> (B, C*H*W)
        x_flat = x.view(batch_size, -1)

        # Fully connected layer適用
        out_flat = self.fc(x_flat)

        # (B, C*H*W) -> (B, C, H, W)
        out = out_flat.view(batch_size, self.num_channels, self.height, self.width)

        return out

class ModelWrapper(nn.Module):
    """
    既存モデルの中間にFC layerを挿入するためのラッパークラス
    """
    def __init__(self, original_model, fc_layer, insert_after_layer):
        super(ModelWrapper, self).__init__()
        self.original_model = original_model
        self.fc_layer = fc_layer
        self.insert_after_layer = insert_after_layer

        # 元のモデルのforwardをフックしてFC layerを中間に挿入
        self._setup_forward_hook()

    def _setup_forward_hook(self):
        """Forward hookを設定してFC layerを中間に挿入"""
        self.activation = {}

        def get_activation(name):
            def hook(model, input, output):
                self.activation[name] = output
            return hook

        # 指定されたレイヤー後にhook設定
        target_layer = self._get_layer_by_name(self.original_model, self.insert_after_layer)
        if target_layer is not None:
            target_layer.register_forward_hook(get_activation(self.insert_after_layer))

    def _get_layer_by_name(self, model, layer_name):
        """レイヤー名で実際のレイヤーオブジェクトを見つける"""
        names = layer_name.split('.')
        layer = model
        for name in names:
            if hasattr(layer, name):
                layer = getattr(layer, name)
            else:
                return None
        return layer

    def forward(self, x):
        # この方法は複雑なので他のアプローチを使用
        pass

def insert_fc_layer_into_model(model, fc_position='middle', input_size=(1200, 1600), init_type='identity'):
    """
    既存モデルにIdentityFullyConnected layerを挿入する関数

    Args:
        model: ロードされた事前学習モデル
        fc_position: FC layer挿入位置 ('middle', 'early', 'late' または特定レイヤー名)
        input_size: 入力画像サイズ (H, W)
        init_type: FC layer初期化方法 ('identity' or 'random')

    Returns:
        新しいモデル (FC layerが挿入された)
    """

    # モデル構造分析
    print("=== 元のモデル構造分析 ===")
    layer_info = analyze_model_structure(model, input_size)

    # FC layer挿入位置決定
    insert_layer_name = determine_insert_position(layer_info, fc_position)
    print(f"FC layer挿入位置: {insert_layer_name}")

    # 挿入位置のfeature mapサイズ確認
    target_layer_info = None
    for info in layer_info:
        if info['name'] == insert_layer_name:
            target_layer_info = info
            break

    if target_layer_info is None:
        raise ValueError(f"Layer {insert_layer_name} not found")

    output_shape = target_layer_info['output_shape']
    num_channels = output_shape[1]
    height = output_shape[2]
    width = output_shape[3]

    print(f"FC layer設定: channels={num_channels}, height={height}, width={width}")

    # IdentityFullyConnected layer生成
    fc_layer = IdentityFullyConnected(num_channels, height, width, init_type)

    # 新しいモデル生成 (FC layer挿入)
    modified_model = create_modified_model(model, fc_layer, insert_layer_name)

    return modified_model, fc_layer

def analyze_model_structure(model, input_size=(1200, 1600)):
    """
    モデル構造を分析して各レイヤーの出力サイズを確認
    """
    model.eval()
    layer_info = []

    # テスト入力生成
    test_input = torch.randn(1, 3, input_size[0], input_size[1])

    # Forward hookを使用して各レイヤーの出力サイズを記録
    def hook_fn(name):
        def hook(module, input, output):
            if isinstance(output, torch.Tensor):
                layer_info.append({
                    'name': name,
                    'module': module,
                    'output_shape': list(output.shape),
                    'output_size': output.numel()
                })
        return hook

    # 全レイヤーにhook登録
    hooks = []
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # leaf moduleのみ
            hook = module.register_forward_hook(hook_fn(name))
            hooks.append(hook)

    # Forward pass実行
    with torch.no_grad():
        _ = model(test_input)

    # Hook削除
    for hook in hooks:
        hook.remove()

    # 結果出力
    print("\nレイヤー別出力サイズ:")
    for i, info in enumerate(layer_info):
        print(f"{i:2d}. {info['name']:<30} : {info['output_shape']} (サイズ: {info['output_size']:,})")

    return layer_info

def determine_insert_position(layer_info, fc_position):
    """
    FC layer挿入位置決定
    """
    if fc_position == 'middle':
        # 中間位置（全レイヤーの中間）
        middle_idx = len(layer_info) // 2
        return layer_info[middle_idx]['name']
    elif fc_position == 'early':
        # 初期位置（全体の1/4地点）
        early_idx = len(layer_info) // 4
        return layer_info[early_idx]['name']
    elif fc_position == 'late':
        # 後半位置（全体の3/4地点）
        late_idx = (len(layer_info) * 3) // 4
        return layer_info[late_idx]['name']
    elif fc_position == 'smallest':
        # 最も小さいfeature mapサイズを持つ位置
        min_size = min(info['output_size'] for info in layer_info)
        for info in layer_info:
            if info['output_size'] == min_size:
                return info['name']
    else:
        # 特定レイヤー名が与えられた場合
        return fc_position

def create_modified_model(original_model, fc_layer, insert_after_layer):
    """
    元のモデルの指定位置にFC layerを挿入した新しいモデル生成
    """

    class ModifiedModel(nn.Module):
        def __init__(self, original_model, fc_layer, insert_after_layer):
            super(ModifiedModel, self).__init__()
            self.original_model = original_model
            self.fc_layer = fc_layer
            self.insert_after_layer = insert_after_layer

            # 挿入位置を見つける
            self.layers_before = nn.ModuleList()
            self.layers_after = nn.ModuleList()
            self.target_layer = None

            self._split_model()

        def _split_model(self):
            """モデルを挿入地点基準で分割"""
            found_target = False

            for name, module in self.original_model.named_modules():
                if len(list(module.children())) == 0:  # leaf moduleのみ
                    if name == self.insert_after_layer:
                        self.target_layer = module
                        found_target = True
                    elif not found_target:
                        self.layers_before.append(module)
                    else:
                        self.layers_after.append(module)

        def forward(self, x):
            # このアプローチは複雑。代わりにより簡単な方法を使用
            return self._forward_with_fc(x)

        def _forward_with_fc(self, x):
            # 元のモデルのforwardを修正してFC layerを挿入
            # これはモデル構造によって異なるので一般的な解決策を提供

            # Hookを使用した方法
            activation = {}

            def get_activation(name):
                def hook(model, input, output):
                    activation[name] = output
                return hook

            # Target layerにhook設定
            target_module = self._get_module_by_name(self.original_model, self.insert_after_layer)
            hook = target_module.register_forward_hook(get_activation('target'))

            # 元のforward実行
            result = self.original_model(x)

            # Hook削除
            hook.remove()

            # 中間結果を得たらFC layer適用後残りの計算
            # これはモデル構造が複雑なので実際にはモデル別カスタマイズが必要

            return result

        def _get_module_by_name(self, model, name):
            """モジュール名で実際のモジュールを見つける"""
            names = name.split('.')
            module = model
            for n in names:
                module = getattr(module, n)
            return module

    # 簡単なラッパーモデル生成
    class SimpleModifiedModel(nn.Module):
        def __init__(self, original_model, fc_layer):
            super(SimpleModifiedModel, self).__init__()
            self.original_model = original_model
            self.fc_layer = fc_layer
            self.insert_point_found = False

        def forward(self, x):
            # これは例であり、実際には特定のモデル構造に合わせて修正が必要
            return self.original_model(x)

    return SimpleModifiedModel(original_model, fc_layer)

def insert_fc_into_real_esrgan(model_path, fc_position='smallest', init_type='identity'):
    """
    Real-ESRGANモデルにFC layerを挿入するメイン関数

    Args:
        model_path: 事前学習モデルファイルパス
        fc_position: FC layer挿入位置
        init_type: 初期化方法

    Returns:
        修正されたモデル、FC layerオブジェクト
    """
    print(f"モデルロード中: {model_path}")

    # モデルロード
    checkpoint = torch.load(model_path, map_location='cpu')

    if isinstance(checkpoint, dict):
        if 'params' in checkpoint:
            model_state = checkpoint['params']
        elif 'state_dict' in checkpoint:
            model_state = checkpoint['state_dict']
        else:
            model_state = checkpoint
    else:
        model = checkpoint

    # モデルオブジェクトではないstate_dictの場合モデル構造再構成が必要
    # ここでは既にモデルオブジェクトだと仮定
    if not isinstance(checkpoint, nn.Module):
        raise ValueError("モデルオブジェクトを直接ロードする必要があります。state_dictではなく全体モデルを保存してください。")

    model = checkpoint

    # FC layer挿入
    modified_model, fc_layer = insert_fc_layer_into_model(
        model, fc_position, input_size=(1200, 1600), init_type=init_type
    )

    return modified_model, fc_layer

def load_and_preprocess_image(image_path, target_size=(1600, 1200)):
    """
    画像をロードして前処理する関数

    Args:
        image_path: 画像ファイルパス
        target_size: ターゲットサイズ (W, H)

    Returns:
        前処理された画像テンソル
    """
    # 画像読み込み
    image = Image.open(image_path).convert('RGB')

    # サイズ調整
    image = image.resize(target_size, Image.LANCZOS)

    # テンソルに変換
    transform = transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet正規化（必要に応じて）
    ])

    image_tensor = transform(image).unsqueeze(0)  # バッチ次元追加

    return image_tensor

def tensor_to_image(tensor):
    """
    テンソルをPIL画像に変換

    Args:
        tensor: 画像テンソル (1, C, H, W)

    Returns:
        PIL Image
    """
    # テンソルからnumpy配列に変換
    if tensor.dim() == 4:
        tensor = tensor.squeeze(0)  # バッチ次元削除

    # [0, 1]範囲にクリップ
    tensor = torch.clamp(tensor, 0, 1)

    # (C, H, W) -> (H, W, C)に変換
    numpy_array = tensor.permute(1, 2, 0).cpu().numpy()

    # [0, 255]に変換
    numpy_array = (numpy_array * 255).astype(np.uint8)

    # PIL Imageに変換
    image = Image.fromarray(numpy_array)

    return image

def process_images_folder(input_folder, output_folder, original_model, modified_model, device='cpu'):
    """
    フォルダ内の全画像を処理してFC layer挿入前後の結果を保存

    Args:
        input_folder: 入力画像フォルダパス
        output_folder: 出力画像フォルダパス
        original_model: 元のモデル
        modified_model: FC layerが挿入されたモデル
        device: 実行デバイス
    """
    # 出力フォルダ作成
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    # サブフォルダ作成
    original_output = output_folder / "original"
    modified_output = output_folder / "with_fc"
    comparison_output = output_folder / "comparison"

    original_output.mkdir(exist_ok=True)
    modified_output.mkdir(exist_ok=True)
    comparison_output.mkdir(exist_ok=True)

    # モデルを評価モードに設定
    original_model.eval()
    modified_model.eval()

    # デバイスに移動
    original_model.to(device)
    modified_model.to(device)

    # サポートされる画像形式
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}

    # 入力フォルダ内の画像ファイル検索
    input_folder = Path(input_folder)
    image_files = [
        f for f in input_folder.iterdir()
        if f.suffix.lower() in supported_formats
    ]

    print(f"\n=== 画像処理開始 ===")
    print(f"入力フォルダ: {input_folder}")
    print(f"出力フォルダ: {output_folder}")
    print(f"発見された画像ファイル数: {len(image_files)}")
    print(f"使用デバイス: {device}")

    for i, image_path in enumerate(image_files):
        print(f"\n処理中 ({i+1}/{len(image_files)}): {image_path.name}")

        try:
            # 画像ロードと前処理
            input_tensor = load_and_preprocess_image(str(image_path), target_size=(1600, 1200))
            input_tensor = input_tensor.to(device)

            with torch.no_grad():
                # 元のモデルで処理
                original_output_tensor = original_model(input_tensor)

                # FC layerが挿入されたモデルで処理
                modified_output_tensor = modified_model(input_tensor)

            # テンソルを画像に変換
            original_image = tensor_to_image(original_output_tensor.cpu())
            modified_image = tensor_to_image(modified_output_tensor.cpu())
            input_image = tensor_to_image(input_tensor.cpu())

            # ファイル名生成
            base_name = image_path.stem

            # 結果画像保存
            original_image.save(original_output / f"{base_name}_original.png")
            modified_image.save(modified_output / f"{base_name}_with_fc.png")

            # 比較画像作成（横並び）
            comparison_image = create_comparison_image(
                input_image, original_image, modified_image,
                titles=["入力", "元のモデル", "FC挿入モデル"]
            )
            comparison_image.save(comparison_output / f"{base_name}_comparison.png")

            # 差分計算
            diff_tensor = torch.abs(original_output_tensor - modified_output_tensor)
            max_diff = diff_tensor.max().item()
            mean_diff = diff_tensor.mean().item()

            print(f"  ✓ 処理完了")
            print(f"    最大差分: {max_diff:.6f}")
            print(f"    平均差分: {mean_diff:.6f}")

        except Exception as e:
            print(f"  ✗ エラー発生: {e}")
            continue

    print(f"\n=== 処理完了 ===")
    print(f"結果は以下のフォルダに保存されました:")
    print(f"  元のモデル結果: {original_output}")
    print(f"  FC挿入モデル結果: {modified_output}")
    print(f"  比較画像: {comparison_output}")

def create_comparison_image(input_img, original_img, modified_img, titles=None):
    """
    3つの画像を横並びで比較画像を作成

    Args:
        input_img: 入力画像
        original_img: 元のモデル結果
        modified_img: 修正されたモデル結果
        titles: 各画像のタイトル

    Returns:
        比較画像
    """
    from PIL import ImageDraw, ImageFont

    # 画像サイズ取得
    width, height = input_img.size

    # 比較画像作成（横並び + タイトル領域）
    title_height = 30
    comparison_width = width * 3
    comparison_height = height + title_height

    comparison_img = Image.new('RGB', (comparison_width, comparison_height), 'white')

    # 画像貼り付け
    comparison_img.paste(input_img, (0, title_height))
    comparison_img.paste(original_img, (width, title_height))
    comparison_img.paste(modified_img, (width * 2, title_height))

    # タイトル追加
    if titles:
        draw = ImageDraw.Draw(comparison_img)
        try:
            # システムフォント使用を試行
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            # デフォルトフォント使用
            font = ImageFont.load_default()

        for i, title in enumerate(titles):
            text_width = draw.textlength(title, font=font)
            x = (width * i) + (width - text_width) // 2
            draw.text((x, 5), title, fill='black', font=font)

    return comparison_img

def test_fc_insertion():
    """FC layer挿入テスト"""
    print("=== FC Layer挿入テスト ===")

    # ダミーモデル生成（テスト用）
    class DummyModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 64, 3, 1, 1)
            self.conv2 = nn.Conv2d(64, 64, 3, 1, 1)
            self.conv3 = nn.Conv2d(64, 32, 3, 1, 1)
            self.conv4 = nn.Conv2d(32, 3, 3, 1, 1)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.relu(self.conv1(x))
            x = self.relu(self.conv2(x))
            x = self.relu(self.conv3(x))
            x = self.conv4(x)
            return x

    dummy_model = DummyModel()

    # FC layer挿入テスト
    try:
        modified_model, fc_layer = insert_fc_layer_into_model(
            dummy_model, 'smallest', (1200, 1600), 'identity'
        )
        print("✓ FC layer挿入成功")

        # テスト実行
        test_input = torch.randn(1, 3, 1200, 1600)
        with torch.no_grad():
            output = modified_model(test_input)
            print(f"✓ Forward pass成功、出力サイズ: {output.shape}")

    except Exception as e:
        print(f"✗ エラー発生: {e}")

def run_image_comparison_demo(input_folder, output_folder, model_path=None):
    """
    画像比較デモ実行

    Args:
        input_folder: 入力画像フォルダ
        output_folder: 出力フォルダ
        model_path: モデルファイルパス（オプション）
    """
    print("=== 画像比較デモ実行 ===")

    # デバイス設定
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用デバイス: {device}")

    if model_path and os.path.exists(model_path):
        # 実際のモデルを使用
        try:
            modified_model, fc_layer = insert_fc_into_real_esrgan(
                model_path, fc_position='smallest', init_type='identity'
            )
            original_model = torch.load(model_path, map_location='cpu')
        except Exception as e:
            print(f"実際のモデルロードに失敗: {e}")
            print("ダミーモデルでデモを実行します...")
            original_model, modified_model = create_dummy_models()
    else:
        print("モデルパスが提供されていないため、ダミーモデルでデモを実行します...")
        original_model, modified_model = create_dummy_models()

    # 画像処理実行
    process_images_folder(
        input_folder=input_folder,
        output_folder=output_folder,
        original_model=original_model,
        modified_model=modified_model,
        device=device
    )

def create_dummy_models():
    """
    デモ用ダミーモデル作成
    """
    class DummyModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
            self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
            self.conv3 = nn.Conv2d(32, 3, 3, 1, 1)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.relu(self.conv1(x))
            x = self.relu(self.conv2(x))
            x = self.conv3(x)
            return torch.clamp(x, 0, 1)

    original_model = DummyModel()

    # FC layerを挿入したモデル作成
    modified_model, fc_layer = insert_fc_layer_into_model(
        copy.deepcopy(original_model), 'middle', (1200, 1600), 'identity'
    )

    return original_model, modified_model

# 使用例
if __name__ == "__main__":
    print("=== Real-ESRGAN FC Layer挿入ツール ===")

    # テスト実行
    test_fc_insertion()

    print("\n=== 使用例 ===")
    print("# 1. 事前学習モデルにFC layer挿入")
    print("model_path = 'path/to/your/real_esrgan_model.pth'")
    print("modified_model, fc_layer = insert_fc_into_real_esrgan(")
    print("    model_path=model_path,")
    print("    fc_position='smallest',  # 'middle', 'early', 'late', 'smallest' または特定レイヤー名")
    print("    init_type='identity'     # 'identity' または 'random'")
    print(")")
    print()
    print("# 2. FC layer重み再初期化")
    print("fc_layer.reinit_weights('random')")
    print()
    print("# 3. 画像フォルダ処理")
    print("run_image_comparison_demo(")
    print("    input_folder='path/to/input/images',")
    print("    output_folder='path/to/output/results',")
    print("    model_path='path/to/model.pth'  # オプション")
    print(")")
    print()
    print("# 4. 直接モデルロード後FC layer挿入")
    print("model = torch.load('your_model.pth')")
    print("modified_model, fc_layer = insert_fc_layer_into_model(")
    print("    model, 'middle', (1200, 1600), 'identity')")
    print()
    print("# 5. 個別画像処理")
    print("process_images_folder(")
    print("    'input_folder', 'output_folder', original_model, modified_model")
    print(")")